In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<div class="alert alert-block alert-warning">
<b>⚠️ Vertex AI Workbench user-managed notebooks is <a href="https://cloud.google.com/vertex-ai/docs/deprecations">deprecated</a>. On January 30, 2025, support for user-managed notebooks will end and the ability to create user-managed notebooks instances will be removed. Existing instances will continue to function but patches, updates, and upgrades won't be available. To continue using Vertex AI Workbench, complete the steps on this page to <a href="https://cloud.google.com/vertex-ai/docs/workbench/user-managed/migrate-to-instances">migrate your user-managed notebooks instances to Vertex AI Workbench instances.</a>⚠️</b>
</div>


# Inventory prediction on ecommerce data using Vertex AI

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/inventory-prediction/inventory_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Open in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2FGoogleCloudPlatform%2Fvertex-ai-samples%2Fmain%2Fnotebooks%2Fofficial%2Fworkbench%2Finventory-prediction%2Finventory_prediction.ipynb">
      <img width="32px" src="https://cloud.google.com/ml-engine/images/colab-enterprise-logo-32px.png" alt="Google Cloud Colab Enterprise logo"><br> Open in Colab Enterprise
    </a>
  </td>    
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/workbench/inventory-prediction/inventory_prediction.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Workbench
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/workbench/inventory-prediction/inventory_prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
</table>


**_NOTE_**: This notebook has been tested in the following environment:

* Python version = 3.9

## Overview

This notebook explores how to build a machine learning model for inventory prediction on an ecommerce dataset. This notebook includes steps for deploying the model on Vertex AI using the Vertex AI SDK and analyzing the deployed model using the What-If Tool. Learn more about [What-If Tool](https://pair-code.github.io/what-if-tool/).

Learn more about [Vertex AI Workbench](https://cloud.google.com/vertex-ai/docs/workbench/introduction) and [Vertex AI Training](https://cloud.google.com/vertex-ai/docs/training/custom-training).

**Note**: The What-IF tool widget is tested on Colab and Vertex AI workbench's managed instances. It may not work on user-managed instances.

### Objective

This tutorial shows you how to do exploratory data analysis, preprocess data, train model, evaluate model, deploy model, configure What-If Tool.

This tutorial uses the following Google Cloud ML services and resources:

- Vertex AI Model
- Vertex AI Endpoint
- Vertex Explainable AI
- Google Cloud Storage
- BigQuery

The steps performed include:

* Load the dataset from BigQuery using the "BigQuery in Notebooks" integration.
* Analyze the dataset.
* Preprocess the features in the dataset.
* Build a random forest classifier model that predicts whether a product is sold in the next 60 days.
* Evaluate the model.
* Deploy the model using Vertex AI.
* Configure and test with the What-If Tool.

### Dataset

The dataset used in this notebook consists of inventory data since 2018 for an ecommerce store. This dataset is publicly available as a BigQuery table named `looker-private-demo.ecomm.inventory_items`, which can be accessed by pinning the `looker-private-demo` project in BigQuery. The table consists of various fields related to ecommerce inventory items such as `id`, `product_id`, `cost`, when the item arrived at the store, and when it was sold. This notebook makes use of the following fields assuming their purpose is as described below:

- `id`: The ID of the inventory item
- `product_id`: The ID of the product
- `created_at`: When the item arrived in the inventory/at the store
- `sold_at`: When the item was sold (*Null if still unsold*)
- `cost`: Cost at which the item was sold
- `product_category`: Category of the product
- `product_brand`: Brand of the product (dropped later as there are too many values)
- `product_retail_price`: Price of the product
- `product_department`: Department to which the product belonged to
- `product_distribution_center_id`: Which distribution center (an approximation of regions) the product was sold from

The dataset is encoded to hide any private information. For example, ID numbers ranging from 1 to 10 are assigned to the distribution centers.

### Costs

This tutorial uses the following billable components of Google Cloud:

- Vertex AI
- BigQuery
- Cloud Storage


Learn about [Vertex AI
pricing](https://cloud.google.com/vertex-ai/pricing), [BigQuery pricing](https://cloud.google.com/bigquery/pricing) and [Cloud Storage
pricing](https://cloud.google.com/storage/pricing), and use the [Pricing
Calculator](https://cloud.google.com/products/calculator/)
to generate a cost estimate based on your projected usage.

## Get started

### Install Vertex AI SDK for Python and other required packages


In [ ]:
! pip3 install --quiet --upgrade google-cloud-aiplatform \
                                    google-cloud-storage \
                                    seaborn \
                                    pandas \
                                    fsspec \
                                    witwidget \
                                    pyarrow \
                                    db-dtypes \
                                    gcsfs \
                                    matplotlib

! pip3 install scikit-learn==1.2 protobuf==3.20.1

### Restart runtime (Colab only)

To use the newly installed packages, you must restart the runtime on Google Colab.

In [ ]:
import sys

if "google.colab" in sys.modules:

    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ The kernel is going to restart. Wait until it's finished before continuing to the next step. ⚠️</b>
</div>


### Authenticate your notebook environment (Colab only)

Authenticate your environment on Google Colab.


In [ ]:
import sys

if "google.colab" in sys.modules:

    from google.colab import auth

    auth.authenticate_user()

### Set Google Cloud project information

To get started using Vertex AI, you must have an existing Google Cloud project. Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}
LOCATION = "us-central1"  # @param {type:"string"}

# set the project id
! gcloud config set project {PROJECT_ID}

### Create a Cloud Storage bucket

Create a storage bucket to store intermediate artifacts such as datasets.

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
! gsutil mb -l {LOCATION} -p {PROJECT_ID} {BUCKET_URI}

### Import libraries

In [ ]:
import os
import pickle

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics
from google.cloud import aiplatform, storage
from google.cloud.bigquery import Client
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from witwidget.notebook.visualization import WitConfigBuilder, WitWidget

### Initialize Vertex AI SDK for Python

Initialize the Vertex AI SDK for Python for your project.

In [ ]:
aiplatform.init(project=PROJECT_ID, location=LOCATION, staging_bucket=BUCKET_URI)

### Load the required data from BigQuery

The following cell integrates with BigQuery data from the same project through the Vertex AI's "BigQuery in Notebooks" integration. It can run an SQL query as it would run in the BigQuery console. 

**Note:** This feature only works in a notebook running on a Vertex AI Workbench managed-notebook instance.

#@bigquery
SELECT 
    id,
    product_id, 
    created_at,
    sold_at,
    cost,
    product_category,
    product_brand,
    product_retail_price,
    product_department,
    product_distribution_center_id
FROM 
looker-private-demo.ecomm.inventory_items

After executing the above cell, clicking **Query and load as DataFrame** button adds the following python cell that loads the queried data into a pandas dataframe.

In [ ]:
# The following two lines are only necessary to run once.
# Comment out otherwise for speed-up.
client = Client(project=PROJECT_ID)

query = """SELECT 
    id,
    product_id, 
    created_at,
    sold_at,
    cost,
    product_category,
    product_brand,
    product_retail_price,
    product_department,
    product_distribution_center_id
FROM 
looker-private-demo.ecomm.inventory_items"""
job = client.query(query)
df = job.to_dataframe()

## Explore and clean the dataset

Check the first five rows of the dataset.

In [ ]:
df.head(5)

Check the fields in the dataset and their data types and number of null values.

In [ ]:
df.info()

Apart from the `sold_at` datetime field, there aren't any fields that consist of null values in the dataset. As you're dealing with the inventory-item data, it's absolutely plausible that there would be some items that haven't been sold yet and hence the null values.

### Clean the datetime fields
Next, convert the date fields to a proper date format to process them in the next steps.

In [ ]:
# convert to proper date columns
df["created_at"] = pd.to_datetime(df["created_at"], format="%Y-%m-%d")
df["sold_at"] = pd.to_datetime(df["sold_at"].dt.strftime("%Y-%m-%d"))

Check the date ranges.

In [ ]:
# check the date ranges
print("Min-sold_at : ", df["sold_at"].min())
print("Max-sold_at : ", df["sold_at"].max())

print("Min-created_at : ", df["created_at"].min())
print("Max-created_at : ", df["created_at"].max())

### Extract useful features

Extract the month from the date field `created_at`.

In [ ]:
# calculate the month when the item has arrived
df["arrival_month"] = df["created_at"].dt.month

Calculate the average number of days a product is in inventory until it's sold.

In [ ]:
# calculate the number of days the item hasn't been sold.
df["shelf_days"] = (df["sold_at"] - df["created_at"]).dt.days

Calculate the discount percentages that apply to the products.

In [ ]:
# calculate the discount offered
df["discount_perc"] = (df["product_retail_price"] - df["cost"]) / df[
    "product_retail_price"
]

### Check the categorical fields
Check the unique products and their brands in the data.

In [ ]:
# check total unique items
df["product_id"].unique().shape, df["product_brand"].unique().shape

The fields `product_id` and `product_brand` seem to have a lot of unique values. For the purpose of prediction, use `product_id` as the primary-key and `product_brand` is dropped as it has too many values/levels. 

Segregate the required numerical and categorical fields to analyze the dataset.

In [ ]:
categ_cols = [
    "product_category",
    "product_department",
    "product_distribution_center_id",
    "arrival_month",
]
num_cols = ["cost", "product_retail_price", "discount_perc", "shelf_days"]

Check the count of individual categories for each categorical field.

In [ ]:
for i in categ_cols:
    print(i, " - ", df[i].unique().shape[0])

Check the distribution of the numerical fields.

In [ ]:
df[num_cols].describe().T

### Visualize the data distributions

Generate bar plots for categorical fields and histograms and box plots for numerical fields to check their distributions in the dataset.

In [ ]:
for i in categ_cols:
    df[i].value_counts(normalize=True).plot(kind="bar")
    plt.title(i)
    plt.show()

for i in num_cols:
    _, ax = plt.subplots(1, 2, figsize=(10, 4))
    df[i].plot(kind="box", ax=ax[0])
    df[i].plot(kind="hist", ax=ax[1])
    ax[0].set_title(i + "-Boxplot")
    ax[1].set_title(i + "-Histogram")
    plt.show()

Most of the fields like discount, department, distribution center-id have a decent distribution. For the field `product_category`, there are some categories that don't constitute 2% of the dataset at least. Although there are outliers in some numerical fields, they are exempted from removing as there can be products that are expensive or belonging to a particular category that doesn't often see many sales. 

## Feature preprocessing

Next, aggregate the data based on suitable categorical fields in the data and take the average number of days it took for the product to get sold. For a given `product_id`, there can be multiple item `id`'s in this dataset and you want to predict at the product level whether that particular product is going to be sold in the next couple of months. You're aggregating the data based on each of the product configurations present in this dataset like the price, cost, category and at which center it's sold. This way the model can predict whether a product with certain properties is going to be sold in the next couple of months.

### Generate aggregate features

For number of days a product got sold in, find the average of the `shelf_days` field.

In [ ]:
groupby_cols = [
    "product_id",
    "product_distribution_center_id",
    "product_category",
    "product_department",
    "arrival_month",
    "product_retail_price",
    "cost",
    "discount_perc",
]
value_cols = ["shelf_days"]


df_prod = df[groupby_cols + value_cols].groupby(by=groupby_cols).mean().reset_index()

Check the aggregated product level data.

In [ ]:
df_prod.head()

Look for null values in the data.

In [ ]:
df_prod.isna().sum() / df.shape[0]

Only the `shelf_days` field has null values that correspond to the `product_id`'s that have no sold items. 

### Plot the data distribution

Plot the distribution of the aggregated `shelf_days` field by generating a box plot.

In [ ]:
df_prod["shelf_days"].plot(kind="box")

Here, you can see that most of the products are sold within 60 days since they've arrived in the inventory/store. In this tutorial, you're going to train a machine learning model that predicts the probability of a product being sold within 60 days.

### Encode the categorical fields

Encode the the `shelf_days` field to generate the target field `sold_in_2mnt` indicating whether the product was sold in 60 days.

In [ ]:
df_prod["sold_in_2mnt"] = df_prod["shelf_days"].apply(
    lambda x: 1 if x >= 0 and x < 60 else 0
)
df_prod["sold_in_2mnt"].value_counts(normalize=True)

Segregate the features into variables for model building.

In [ ]:
target = "sold_in_2mnt"
categ_cols = [
    "product_category",
    "product_department",
    "product_distribution_center_id",
    "arrival_month",
]
num_cols = ["product_retail_price", "cost", "discount_perc"]

Encode the `product_department` field.

In [ ]:
df["product_deprtment"] = (
    df["product_department"].apply(lambda x: 1 if x == "Women" else 0).value_counts()
)

Encode the rest of the categorical fields for model building.

In [ ]:
# Create dummy variables for each categ. variable
for i in categ_cols:
    ml = pd.get_dummies(df_prod[i], prefix=i + "_", drop_first=True)
    df_new = pd.concat([df_prod, ml], axis=1)

df_new.drop(columns=categ_cols, inplace=True)
df_new.shape

### Normalize the numerical fields

Normalize the fields `product_retail_price` and `cost` to the 0-1 scale using Min-Max normalization technique.

In [ ]:
scaler = MinMaxScaler()
scaler = scaler.fit(df_new[["product_retail_price", "cost"]])
df_new[["product_retail_price_norm", "cost_norm"]] = scaler.transform(
    df_new[["product_retail_price", "cost"]]
)

## Train the model

Collect the required fields from the dataframe.

In [ ]:
cols = [
    "discount_perc",
    "arrival_month__2",
    "arrival_month__3",
    "arrival_month__4",
    "arrival_month__5",
    "arrival_month__6",
    "arrival_month__7",
    "arrival_month__8",
    "arrival_month__9",
    "arrival_month__10",
    "arrival_month__11",
    "arrival_month__12",
    "product_retail_price_norm",
    "cost_norm",
]

Split the data into train(80%) and test(20%) sets.

In [ ]:
X = df_new[cols].copy()
y = df_new[target].copy()
train_X, test_X, train_y, test_y = train_test_split(
    X, y, train_size=0.8, test_size=0.2, random_state=7
)

Create a [Random Forest classifier](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) object and fit it on the training data.

In [ ]:
model = RandomForestClassifier(random_state=7, n_estimators=100)
model.fit(train_X[cols], train_y)

## Evaluate the model

Predict on the test set and check the accuracy of the model.

In [ ]:
pred_y = model.predict(test_X[cols])

# Calculate the accuracy as our performance metric
accuracy = metrics.accuracy_score(test_y, pred_y)
print("Accuracy: ", accuracy)

Generate the confusion-matrix on the test set.

In [ ]:
confusion = metrics.confusion_matrix(test_y, pred_y)
print(f"Confusion matrix:\n{confusion}")

print("\nNormalized confusion matrix:")
for row in confusion:
    print(row / row.sum())

The model performance can be stated in terms of specificity (True-negative rate) and sensitivity (True-positive rate). In the normalized confusion matrix, the top left value represents the True-negative rate and the bottom right value represents the True-positive rate.

## Save the model to a Cloud Storage bucket

Next, save the model to the created Cloud Storage bucket for deployment.

In [ ]:
# save the trained model to a local file "model.pkl"
FILE_NAME = "model.pkl"
with open(FILE_NAME, "wb") as file:
    pickle.dump(model, file)

# Upload the saved model file to Cloud Storage
BLOB_PATH = "inventory_prediction/"
BLOB_NAME = os.path.join(BLOB_PATH, FILE_NAME)

bucket = storage.Client().bucket(BUCKET_URI[5:])

blob = bucket.blob(BLOB_NAME)
blob.upload_from_filename(FILE_NAME)

## Upload the model to Vertex AI

Specify the following parameters to create a model in Vertex AI Model Registry:

- `display_name`: The display name of the Model.
- `artifact_uri`: The path to the directory containing the Model artifact and any of its supporting files.
- `serving_container_image_uri`: The URI of the Model serving container.

Learn more about [Vertex AI Model Registry](https://cloud.google.com/vertex-ai/docs/model-registry/introduction).

In [ ]:
MODEL_DISPLAY_NAME = "inventory-pred-model-unique"  # @param {type:"string"}
ARTIFACT_GCS_PATH = f"{BUCKET_URI}/{BLOB_PATH}"

Create a Vertex AI model resource.

Ensure that the Sklearn's version for the serving container matches with the local version used for training the model. Learn more about the available [pre-built containers for Vertex AI](https://cloud.google.com/vertex-ai/docs/predictions/pre-built-containers).

In [ ]:
model = aiplatform.Model.upload(
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=ARTIFACT_GCS_PATH,
    serving_container_image_uri="us-docker.pkg.dev/vertex-ai/prediction/sklearn-cpu.1-2:latest",
)

model.wait()

print("Display name:\n", model.display_name)
print("Resource name:\n", model.resource_name)

## Create a Vertex AI Endpoint

Set the display name for the endpoint.

In [ ]:
ENDPOINT_DISPLAY_NAME = "inventory-pred-endpoint-unique"  # @param {type:"string"}

Create an endpoint resource on Vertex AI.

In [ ]:
endpoint = aiplatform.Endpoint.create(display_name=ENDPOINT_DISPLAY_NAME)

print("Display name:\n", endpoint.display_name)
print("Resource name:\n", endpoint.resource_name)

## Deploy the model to the created endpoint

Specify the machine type needed for serving the deployed model.

In [ ]:
MACHINE_TYPE = "n1-standard-2"

Deploy the model to the created endpoint.

In [ ]:
model.deploy(endpoint=endpoint, machine_type=MACHINE_TYPE)

model.wait()

print("Model display-name:\n", model.display_name)
print("Model resource-name:\n", model.resource_name)

List the models deployed to the endpoint and ensure that the inventory prediction model is listed.

In [ ]:
endpoint.list_models()

## What-If Tool

The What-If Tool can be used to analyze the model predictions on test data. In this tutorial, the What-If Tool is configured and run on the model deployed on Vertex AI Endpoints in the previous steps.

`WitConfigBuilder` provides the `set_ai_platform_model()` method to configure the What-If Tool with a model deployed as a version on Ai Platform models. This feature currently supports only Ai Platform but not Vertex AI models. Fortunately, there's also an option to pass a custom function for generating predictions through the `set_custom_predict_fn()` method where either the locally trained model or a function that returns predictions from a Vertex AI model can be passed.

Learn more about [What-If Tool](https://pair-code.github.io/what-if-tool/get-started/).

### Prepare test samples

Save some samples from the test data for both the available classes (Fraud/not-Fraud) to analyze the model using the What-If Tool.

In [ ]:
# collect some samples for each class-label from the test data
sample_size = 200
pos_samples = test_y[test_y == 1].sample(sample_size).index
neg_samples = test_y[test_y == 0].sample(sample_size).index
test_samples_y = pd.concat([test_y.loc[pos_samples], test_y.loc[neg_samples]])
test_samples_X = test_X.loc[test_samples_y.index].copy()

### Run the What-If Tool on the deployed Vertex AI model

Define a function to fetch the predictions from the deployed model and run it on the created test data configuring the What-If tool.

In [ ]:
# configure the target and class-labels
TARGET_FEATURE = target
LABEL_VOCAB = ["not-sold", "sold"]

# function to return predictions from the deployed Model


def endpoint_predict_sample(instances: list):
    prediction = endpoint.predict(instances=instances)
    preds = [[1 - i, i] for i in prediction.predictions]
    return preds


# Combine the features and labels into one array for the What-If Tool
test_examples = np.hstack(
    (test_samples_X.to_numpy(), test_samples_y.to_numpy().reshape(-1, 1))
)

# Configure the WIT with the prediction function
config_builder = (
    WitConfigBuilder(test_examples.tolist(), test_samples_X.columns.tolist() + [target])
    .set_custom_predict_fn(endpoint_predict_sample)
    .set_target_feature(TARGET_FEATURE)
    .set_label_vocab(LABEL_VOCAB)
)

# run the WIT-widget
WitWidget(config_builder, height=800)

### Understanding the What-If tool

In the **Datapoint editor** tab, you can highlight a dot in the result set and ask the What-If Tool to pick the "nearest counterfactual". This is a row of data closest to the row of data you selected but with the opposite outcome. Features in the left-hand table are editable and can show what tweaks are needed to get a particular row of data to flip from one outcome to another. For example, altering the *discount_percentage* feature would show how it impacts the prediction. 

<img src="images/Datapoint_editor.png">

Under the **Performance & Fairness** tab, you can slice the prediction results by a second variable. This allows digging deeper and understanding how different segments of the data react to the model's predictions.  For example, in the following image, the higher the *discount_percentage*, the lesser the false negatives and the lower the *discount_percentage*, the higher the false positives. 

<img src="images/Performance_and_fairness.png">

The **Features** tab in the end provides you an intuitive and interactive way to understand the features present in the data. Similar to the exploratory data analysis steps performed in this notebook, the What-If Tool provides a visual and statistical description on the features.

<img src="images/features.PNG">

## Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Vertex AI Endpoint
- Vertex AI Model
- Cloud Storage bucket (set `delete_bucket` to *True* for deletion)

In [ ]:
# Undeploy the model
endpoint.undeploy_all()

# Delete the endpoint
endpoint.delete()

# Delete the model
model.delete()

# Delete locally generated files
! rm -rf model.pkl

# Set this to true only if you'd like to delete your bucket
delete_bucket = False

if delete_bucket:
    ! gsutil -m rm -r $BUCKET_URI